In [1]:
import datetime as dt # date and time tools

from collections import namedtuple

from os import path

import numpy as np
from scipy.integrate import solve_ivp
import networkx as nx

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import io, base64
from IPython.display import Image, HTML

# alguns textos
aa

In [2]:
#beta, theta, p, lamb, sigma, rho, epsA, gammaA, epsI, gammaI, dI, gammaD, dD

In [3]:
num_pop = 300
beta = 0.5
theta = 0.2
p = 0.1
lamb = 0.2
sigma = 0.2
rho = 0.5
epsA = 0.1
gammaA = 0.4
epsI = 0.3
gammaI = 0.2
dI = 0.1
gammaD = 0.2
dD = 0.2
I_0 = 8
T = 80
tempos = np.array(range(T+1))
num_sim = 60

In [4]:
pop_0 = np.ones(num_pop)
infectados_0 = np.random.choice(num_pop, I_0)
pop_0[infectados_0] = 3*np.ones(I_0)
attr_pop_0 = dict([(i, {'estado': int(pop_0[i])}) for i in range(num_pop)])
print(pop_0)

[1. 3. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 3. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 3. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 3. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 3. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 3. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 3. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 3. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [5]:
def evolucao_SEIRQAD(pop_0, beta, theta, p, lamb, sigma, rho, epsA, gammaA, epsI, gammaI, dI, gammaD, dD, tempos, plot=False):

    SEIRQAD_Compartimental = namedtuple('SEIRQAD_Compartimental', 
                                 [
                                     'pop_0',
                                     'num_pop',
                                     'beta',
                                     'theta',
                                     'p',
                                     'lamb',
                                     'sigma',
                                     'rho',
                                     'epsA',
                                     'gammaA',
                                     'epsI',
                                     'gammaI',
                                     'dI',
                                     'gammaD',
                                     'dD',
                                     't_instantes',
                                     'S',
                                     'E'
                                     'I', 
                                     'R',
                                     'Q',
                                     'A',
                                     'D',
                                     'X'
                                 ])        
          
    def diferencial(t, v, beta, theta, p, lamb, sigma, rho, epsA, gammaA, epsI, gammaI, dI, gammaD, dD):
        """Differential equation for SEIR-QAD model"""
        S,Q,E,A,I,D,R = v
        dS = - beta*S*(I + theta*A) - p*S + lamb*Q
        dQ = p*S - lamb*Q
        dE = beta*S*(I + theta*A) - sigma*E
        dA = sigma*(1 - rho)*E - epsA*A - gammaA*A
        dI = sigma*rho*E - epsI*I - gammaI*I - dI*I
        dDt = epsA*A + epsI*I - gammaD*D - dD*D
        dR = gammaA*A + gammaI*I + gammaD*D
        return np.array([dS, dQ, dE, dA, dI, dDt, dR])   
    
    y0 = pop_0[0:7]
    sol = solve_ivp(diferencial, t_span=[tempos[0],tempos[-1]], 
                    y0 = y0, t_eval = tempos,
                    args=(beta, theta, p, lamb, sigma, rho, epsA, gammaA, epsI, gammaI, dI, gammaD, dD))
    
    resultado = SEIRQAD_Compartimental(pop_0, num_pop, beta, theta, p, lamb, sigma, rho, epsA, gammaA, epsI, gammaI,
                                       dI, gammaD, dD, tempos,
                                       sol.y[0], sol.y[1], sol.y[2], sol.y[3], sol.y[4], sol.y[5], sol.y[6], sol)
    
    return resultado

In [6]:
def analise_grafo(G, info=True, node_size=0, pos=None, hist=False):

    num_vertices = G.number_of_nodes()
    num_arestas = G.number_of_edges()
    num_arestas_por_vertice = [j for i,j in G.degree]
    num_medio_conexoes = 2*num_arestas/num_vertices

    if info:
        print(f'Número de arestas: {num_arestas}')
        print(f'Número de vértices: {num_vertices}')
        print(f'Número médio de conexões por indivíduo: {num_medio_conexoes:.1f}')

    if node_size:
        color = ['tab:blue', 'tab:red', 'tab:green', 'tab:orange', 'tab:pink', 'tab:purple', 'tab:brown']
        pop_estado = nx.get_node_attributes(G,'estado')
        color_map = [color[pop_estado[j]-1] for j in range(num_vertices)]
        plt.figure(figsize=(10,6))
        if pos:
            nx.draw(G, pos, node_size=node_size, node_color=color_map, alpha=0.5)
        else:
            nx.draw(G, node_size=node_size, node_color=color_map, alpha=0.5)
        plt.title('Rede de indivíduos e de suas conexões', fontsize=16)
        plt.show()

    if hist:
        plt.figure(figsize=(10,6))
        plt.hist(num_arestas_por_vertice, 50, facecolor='g', alpha=0.75)
        plt.xlabel('num. arestas', fontsize=14)
        plt.ylabel('num. vertices', fontsize=14)
        plt.title('Histograma do número vertices por número de arestas', fontsize=16)
        plt.show()
    return num_medio_conexoes

In [7]:
# beta,          theta,                  p,             lamb,         sigma,            rho,           epsA,                 gammaA,                 epsI,              gammaI,               dI,           gammaD,                     dD
# prob_infeccao, prob_infeccao_assint, prob_entrar_quar, prob_sair_quar, prob_incubacao, proporcao_inf, prob_diagnost_assint, prob_recperacao_assint, prob_diagnost_inf, prob_recuperacao_inf, prob_obit_inf, prob_recuperacao_diagnost, prob_obit_diagnost

In [8]:
def evolucao_grafo_estruturado(pop_0, prob_infeccao, prob_infeccao_assint, prob_entrar_quar, prob_sair_quar, prob_incubacao, proporcao_inf, prob_diagnost_assint, prob_recperacao_assint, prob_diagnost_inf, prob_recuperacao_inf, prob_obit_inf, prob_recuperacao_diagnost, prob_obit_diagnost, G, tempos, num_sim, show=''):


    def passo_grafo_estruturado(num_pop, populacao, A_adj, prob_infeccao, prob_infeccao_assint, prob_entrar_quar, prob_sair_quar, prob_incubacao, proporcao_inf, prob_diagnost_assint, prob_recperacao_assint, prob_diagnost_inf, prob_recuperacao_inf, prob_obit_inf, prob_recuperacao_diagnost, prob_obit_diagnost):
       
        I = np.eye(num_pop)
        # gera uma matriz cheia aleatória (números em [0.0,1.0))
        A_random = np.random.rand(num_pop, num_pop)
        
        # filtra de acordo com as conexões
        A_graph_random = np.multiply(A_adj, A_random)
        
        # copia a população (acho que é desnecessário)
        pop_aux = np.copy(populacao)
        
        # separa os suscetíveis, criando um vetor de 1's e 0's, se for, ou não, suscetível
        pop_suscetiveis = np.select([pop_aux==1], [pop_aux])
        
        # separa os infectados, criando um vetor de 1's e 0's, se for, ou não, infectado
        pop_infectados = np.select([pop_aux==3], [pop_aux])/3
        
        # separa os assintomaticos, criando um vetor de 1's e 0's, se for, ou não, infectado assintomático
        pop_assintomaticos = np.select([pop_aux==4], [pop_aux])/4
        
        # cria uma matriz de risco, mantendo apenas as conexões que envolvem um infectado
        A_risco = np.multiply(np.tile(pop_infectados, (num_pop, 1)), A_graph_random)
        
        # cria uma matriz de risco, mantendo apenas as conexões que envolvem um infectado assintomatico
        B_risco = np.multiply(np.tile(pop_assintomaticos, (num_pop, 1)), A_graph_random)
        
        # cria matriz de contatos entre um suscetível e um infectado
        A_contatos = np.multiply(np.tile(pop_suscetiveis, (num_pop, 1)).transpose(), A_risco)
        
        # cria matriz de contatos entre um suscetível e um assintomatico
        B_contatos = np.multiply(np.tile(pop_suscetiveis, (num_pop, 1)).transpose(), B_risco)
        
        # cria uma matriz de 1's e 0's, indicando se houve, ou não, contágio por um infectado
        A_infectados = np.select([A_contatos > 1-prob_infeccao], [np.ones([num_pop, num_pop])])
        
        # cria uma matriz de 1's e 0's, indicando se houve, ou não, contágio por um assintomatico
        B_infectados = np.select([B_contatos > 1-prob_infeccao_assint], [np.ones([num_pop, num_pop])])

        # obtém novos expostos
        pop_novos_expostos = np.select([np.sum(A_infectados + B_infectados, axis=1)>0], [np.ones(num_pop)])
        
        # separa os expostos, criando um vetor de 1's e 0's, se for, ou não, exposto
        pop_expostos = np.select([pop_aux==2], [pop_aux])/2
        
        # separa os diagnosticados, criando um vetor de 1's e 0's, se for, ou não, diagnosticado
        pop_diagnosticados = np.select([pop_aux==5], [pop_aux])/5
        
        # filtra matriz aleatória com a diagonal
        pop_recuperando_inf = pop_infectados @ np.multiply(I, A_random)
        pop_recuperando_assint = pop_assintomaticos @ np.multiply(I, A_random)
        pop_recuperando_diagnost = pop_diagnosticados @ np.multiply(I, A_random)
        pop_incubacao = pop_expostos @ np.multiply(I, A_random)
 
        
        # obtém novos recuperados
        pop_novos_recuperados_inf = np.select([pop_recuperando_inf > 1-prob_recuperacao_inf], [np.ones(num_pop)])
        pop_novos_recuperados_assint = np.select([pop_recuperando_assint > 1-prob_recuperacao_assint], [np.ones(num_pop)])
        pop_novos_recuperados_diagnost = np.select([pop_recuperando_diagnost > 1-prob_recuperacao_diagnost], [np.ones(num_pop)])
        
        pop_novos_recuperados = pop_novos_recuperados_inf + pop_novos_recuperados_assint + pop_novos_recuperados_diagnost
        
        # obtém novos infectados e assintomáticos
        pop_novos_infs = np.multiply(np.select([pop_incubacao > 1-prob_incubacao], [np.ones(num_pop)]), np.random.rand(num_pop))
        pop_novos_infectados = np.select([pop_novos_infs < proporcao_inf], np.ones(num_pop))
        pop_novos_assintomaticos = np.select([pop_novos_infs >= proporcao_inf], [np.ones(num_pop)])
        
        # separa os suscetíveis, criando um vetor de 1's e 0's, se for, ou não, exposto
        pop_suscetiveis = np.select([pop_aux==1], [pop_aux])
        
        # separa os quarentenados, criando um vetor de 1's e 0's, se for, ou não, exposto
        pop_quarentenados = 1 + np.select([pop_aux==0], [pop_aux])
        
        # filtra matriz aleatória com a diagonal
        pop_quarentenando = pop_suscetiveis @ np.multiply(I, A_random)
        pop_saindo_quar = pop_quarentenados @ np.multiply(I, A_random)

        # obtém novos quarentenados
        pop_novos_quarentenados = np.select([np.logical_and(pop_quarentenando > 1-prob_entrar_quar, pop_novos_expostos == 0)], [np.ones(num_pop)])
        
        # obtém novos suscetíveis
        pop_novos_suscetiveis = np.select([pop_saindo_quar <1 - prob_sair_quar], [np.ones(num_pop)])
        
        # obtém novos diagnosticados
        diagnost_random = np.random.rand(num_pop)
        pop_diagnosticando_inf = np.multiply(pop_infectados, diagnost_random)
        pop_diagnosticando_assint = np.multiply(pop_assint, diagnost_random)
        pop_novos_diagnost_inf = np.select([np.logical_and(pop_diagnost_inf > 1-prob_diagnost_inf, pop_novos_recuperados == 0)], [np.ones(num_pop)])
        pop_novos_diagnost_assint = np.select([np.logical_and(pop_diagnost_assint > 1-prob_diagnost_assint, pop_novos_recuperados == 0)], [np.ones(num_pop)])
        pop_novos_diagnosticados = pop_novos_diagnost_inf + pop_novos_diagnost_assint
        
        # obtém novos obitos
        obitos_random = np.random.rand(num_pop)
        pop_obit_inf = np.multiply(pop_infectados, obitos_random)
        pop_obit_diagnost = np.multiply(pop_diagnosticados, obitos_random)
        pop_novos_obitos_inf = np.select([np.logical_and(pop_obit_inf > 1-prob_obit_inf, pop_novos_recuperados == 0, pop_novos_diagnosticados == 0)], [np.ones(num_pop)])
        pop_novos_obitos_diagnost = np.select([np.logical_and(pop_obit_diagnost > 1-prob_obit_diagnost, pop_novos_recuperados == 0)], [np.ones(num_pop)])
        pop_novos_obitos = pop_novos_obitos_inf + pop_novos_obitos_diagnost
        
        # calculo das novas populações de suscetiveis(considerando os que permanecem no compartimento):
        ind_susc = np.logical_or(np.logical_and(pop_novos_quarentenados == 0, pop_novos_expostos == 0, pop_aux == 1), pop_novos_suscetiveis == 1)
        suscetiveis = np.select([ind_susc], [np.ones(num_pop)])
        
        # calculo das novas populações de expostos(considerando os que permanecem no compartimento):
        ind_exp = np.logical_or(np.logical_and(pop_novos_infectados == 0, pop_novos_assintomaticos == 0, pop_aux == 2), pop_novos_expostos == 2)
        expostos = np.select([ind_exp], [np.ones(num_pop)])
        
        # calculo das novas populações de infectados(considerando os que permanecem no compartimento):
        ind_inf = np.logical_or(np.logical_and(pop_novos_obitos == 0, pop_novos_diagnosticados == 0, pop_novos_recuperados == 0, pop_aux == 3), pop_novos_infectados ==1)
        expostos = np.select([ind_inf], [np.ones(num_pop)]) 
        
        # calculo das novas populações de assintomaticos(considerando os que permanecem no compartimento):
        ind_assint = np.logical_or(np.logical_and(pop_novos_diagnosticados == 0, pop_novos_recuperados == 0, pop_aux == 4), pop_novos_assintomaticos==1)
        assintomaticos = np.select([ind_assint], [np.ones(num_pop)]) 
        
        # calculo das novas populações de diagnosticados(considerando os que permanecem no compartimento):
        ind_diag = np.logical_or(np.logical_and(pop_novos_recuperados == 0, pop_novos_obitos == 0, pop_aux == 5), pop_novos_diagnosticados==1)
        diagnosticados = np.select([ind_diag], [np.ones(num_pop)])
        
        # calculo das novas populações de recuperados (considerando os que permanecem no compartimento):
        ind_rec = np.logical_or(pop_aux == 6, pop_novos_recuperados==1)
        recuperados = np.select([ind_rec], [np.ones(num_pop)])
        
        # calculo de novos obitos (considerando os que permanecem no compartimento):
        ind_ob = np.logical_or(pop_aux == -1, pop_novos_obitos==1)
        obitos = np.select([ind_ob], [np.ones(num_pop)])
        
        
        # atualiza população adicionando um aos que avançaram de estágio
        populacao_nova = suscetiveis + 2*expostos + 3*infectados + 4*assintomaticos + 5*diagnosticados + 6*recuperados + (-1)*obitos
        

        # Observe que cada elemento da matriz aleatória é usado apenas uma vez, garantindo
        # a independência desses eventos aleatórios (tanto quanto se leve em consideração
        # que os números gerados são pseudo-aleatórios)

        return populacao_nova

    
    # confere se escolha para `show` é válida
    if show:
        assert(show in ('nuvem', 'sd', 'media')), 'Valor inválido para argumento `show`.'

        
    # atributos de saida   
    SEIRQAD_Individual = namedtuple('SEIRQAD_Individual', 
                                 [
                                    'pop_0',
                                    'num_pop',
                                    'prob_infeccao',
                                    'prob_infeccao_assint', 
                                    'prob_entrar_quar', 
                                    'prob_sair_quar', 
                                    'prob_incubacao', 
                                    'proporcao_inf', 
                                    'prob_diagnost_assint', 
                                    'prob_recperacao_assint', 
                                    'prob_diagnost_inf', 
                                    'prob_recuperacao_inf', 
                                    'prob_obit_inf', 
                                    'prob_recuperacao_diagnost', 
                                    'prob_obit_diagnost',
                                    'G',
                                    'tempos',
                                    'num_sim',
                                    'S_mean',
                                    'E_mean', 
                                    'I_mean',
                                    'R_mean',
                                    'Q_mean',
                                    'A_mean',
                                    'D_mean',
                                    'O_mean',
                                    'S_sigma',
                                    'E_sigma', 
                                    'I_sigma',
                                    'R_sigma', 
                                    'Q_sigma', 
                                    'A_sigma', 
                                    'D_sigma', 
                                    'O_sigma'
                                ])
    
    # número de indivíduos da população
    num_pop = len(pop_0)
    I_0 = np.count_nonzero(pop_0==3) 
   
    # número de instantes no tempo e passos de tempo 
    num_t = len(tempos)
    passos_de_tempo = tempos[1:] - tempos[:-1]

    # inicializa variáveis para o cálculo da média
    S_mean = np.zeros(num_t)
    E_mean = np.zeros(num_t)
    I_mean = np.zeros(num_t)
    R_mean = np.zeros(num_t)
    Q_mean = np.zeros(num_t)
    A_mean = np.zeros(num_t)
    D_mean = np.zeros(num_t)
    O_mean = np.zeros(num_t)

    # inicializa variáveis para o cálculo do desvio padrão
    S_sigma = np.zeros(num_t)
    E_sigma = np.zeros(num_t)
    I_sigma = np.zeros(num_t)
    R_sigma = np.zeros(num_t)
    Q_sigma = np.zeros(num_t)
    A_sigma = np.zeros(num_t)
    D_sigma = np.zeros(num_t)
    O_sigma = np.zeros(num_t)
    
    # obtém matriz de adjacências e o número médio de vizinhos do grafo
    A_adj = nx.to_numpy_matrix(G)

    # prepara gráfico se necessário
    if show in ('nuvem', 'sd', 'media'):    
        # inicializa figura e define eixo vertical
        plt.figure(figsize=(12,6))
        plt.ylim(0, num_pop)
        plt.xlim(tempos[0], tempos[-1])
    
    if show == 'nuvem':
        # alpha para a nuvem de gráficos das diversas simulaçõe
        alpha = min(0.2, 5/num_sim)    

    # simulações
    for j in range(num_sim):

        # inicializa população de cada simulação
        populacao = np.copy(pop_0)
        S = np.array([num_pop - I_0])
        E = np.array([0]) 
        I = np.array([I_0])
        R = np.array([0])
        Q = np.array([0])
        A = np.array([0]) 
        D = np.array([0])
        O = np.array([0])
     
  
        # evolui o dia e armazena as novas contagens
        for dt in passos_de_tempo:

            # calcula probabilidades
            prob_beta = 1 - np.exp(-prob_infeccao*dt) 
            prob_theta = 1 - np.exp(-prob_infeccao_assint*dt) 
            prob_p = 1 - np.exp(-prob_entrar_quar*dt) 
            prob_lamb = 1 - np.exp(-prob_sair_quar*dt) 
            prob_sigma = 1 - np.exp(-prob_incubacao*dt) 
            prob_rho = 1 - np.exp(-proporcao_inf*dt)
            prob_epsA = 1 - np.exp(-prob_diagnost_assint*dt) 
            prob_gammaA = 1 - np.exp(-prob_recuperacao_assint*dt) 
            prob_epsI = 1 - np.exp(-prob_diagnost_inf*dt) 
            prob_gammaI = 1 - np.exp(-prob_recuperacao_inf*dt) 
            prob_dI = 1 - np.exp(-prob_obit_inf*dt) 
            prob_gammaD = 1 - np.exp(-prob_recuperacao_diagnost*dt) 
            prob_dD = 1 - np.exp(-prob_obit_diagnost*dt) 
            
            populacao = passo_grafo_estruturado(num_pop, populacao, A_adj, 
                                                prob_beta, prob_theta, prob_p, prob_lamb, prob_sigma, prob_rho, 
                                                prob_epsA, prob_gammaA, prob_epsI, prob_gammaI, prob_dI, prob_gammaD, prob_dD)

            
            Q = np.hstack([Q, np.count_nonzero(populacao==0)])
            S = np.hstack([S, np.count_nonzero(populacao==1)])
            E = np.hstack([E, np.count_nonzero(populacao==2)])
            I = np.hstack([I, np.count_nonzero(populacao==3)])
            A = np.hstack([A, np.count_nonzero(populacao==4)])
            D = np.hstack([D, np.count_nonzero(populacao==5)])
            R = np.hstack([R, np.count_nonzero(populacao==6)])
            O = np.hstack([O, np.count_nonzero(populacao==-1)])
            
        # adiciona as contagens dessa simulação para o cálculo final da média
        S_mean += S
        E_mean += E
        I_mean += I
        R_mean += R
        Q_mean += Q
        A_mean += A
        D_mean += D
        O_mean += O
        

        # adiciona as contagens dessa simulação para o cálculo final do desvio padrão
        S_sigma += S ** 2
        E_sigma += E ** 2
        I_sigma += I ** 2
        R_sigma += R ** 2
        Q_sigma += Q ** 2
        A_sigma += A ** 2
        D_sigma += D ** 2
        O_sigma += O ** 2
        
        colors = ['tab:blue', 'tab:red', 'tab:green', 'tab:orange', 'tab:pink', 'tab:purple', 'tab:brown', 'black']
        comps = [S, E, I, R, Q, A, D, O]
        nomes = ['Suscetiveis', 'Expostos', 'Infectados', 'Recuperados', 'Quarentenados', 'Assintomáticos', 'Diagnosticados', 'Óbitos']
        if show == 'nuvem':
            # exibe os gráficos dos dados de cada simulação
            for i in range(0,8):
                plt.plot(tempos, comps[i], '-', color= colors[i], alpha=alpha)
            #plt.plot(tempos, num_pop - S, '-', color='tab:gray', alpha=alpha)

    # divide pelo número de evoluções para obter a média
    S_mean /= num_sim
    E_mean /= num_sim
    I_mean /= num_sim
    R_mean /= num_sim
    Q_mean /= num_sim
    A_mean /= num_sim
    D_mean /= num_sim
    O_mean /= num_sim

    # ajusta o calcula do desvio padrão
    S_sigma = ( S_sigma / num_sim - S_mean**2 )**.5
    E_sigma = ( E_sigma / num_sim - E_mean**2 )**.5
    I_sigma = ( I_sigma / num_sim - I_mean**2 )**.5
    R_sigma = ( R_sigma / num_sim - R_mean**2 )**.5
    Q_sigma = ( Q_sigma / num_sim - Q_mean**2 )**.5
    A_sigma = ( A_sigma / num_sim - A_mean**2 )**.5
    D_sigma = ( D_sigma / num_sim - D_mean**2 )**.5
    O_sigma = ( O_sigma / num_sim - O_mean**2 )**.5
    
    comps_mean = [S_mean, E_mean, I_mean, R_mean, Q_mean, A_mean, D_mean, O_mean]
    comps_sigma = [S_sigma, E_sigma, I_sigma, R_sigma, Q_sigma, A_sigma, D_sigma, O_sigma]
    # exibe os gráficos das médias
    if show in ('nuvem', 'sd', 'media'):
        for i in range(0,8):
            plt.plot(tempos, comps_mean[i], '-', color=colors[i], label=nomes[i])
        #plt.plot(tempos, num_pop - S_mean, '-', color='tab:gray', label='inf.+ rec.')
        
    if show == 'sd':
        for i in range(0,8):
            plt.fill_between(tempos, comps_mean[i] - comps_sigma[i], comps_mean[i] + comps_sigma[i], facecolor=colors[i], alpha = 0.2)
        #plt.fill_between(tempos, num_pop - S_mean - S_sigma, num_pop - S_mean + S_sigma, facecolor='tab:gray', alpha = 0.2)

        
    if show == 'nuvem':
        plt.title('Evolução do conjunto de simulações e da média', fontsize=16)
    elif show == 'sd':
        plt.title('Evolução da média, com o desvio padrão', fontsize=16)
    elif show == 'media':
        plt.title('Evolução da média das simulações', fontsize=16)

        
    # informações para o gráfico
    if show in ('nuvem', 'sd', 'media'):
        plt.xlabel('tempo', fontsize=14)
        plt.ylabel('número de indivíduos', fontsize=14)
        plt.legend(fontsize=12)
        plt.show() 

    resultado = SEIRQAD_Individual(
                                pop_0,
                                num_pop,
                                prob_infeccao,
                                prob_infeccao_assint, 
                                prob_entrar_quar, 
                                prob_sair_quar, 
                                prob_incubacao, 
                                proporcao_inf, 
                                prob_diagnost_assint, 
                                prob_recperacao_assint, 
                                prob_diagnost_inf, 
                                prob_recuperacao_inf, 
                                prob_obit_inf, 
                                prob_recuperacao_diagnost, 
                                prob_obit_diagnost,
                                G,
                                tempos,
                                num_sim,
                                S_mean,
                                E_mean, 
                                I_mean,
                                R_mean,
                                Q_mean,
                                A_mean,
                                D_mean,
                                O_mean,
                                S_sigma,
                                E_sigma, 
                                I_sigma,
                                R_sigma, 
                                Q_sigma, 
                                A_sigma, 
                                D_sigma, 
                                O_sigma
                                )
        
        
    
    return resultado

In [9]:
X = evolucao_SEIRQAD([num_pop - I_0, 0, I_0] + 4*[0], beta , theta, p, lamb, sigma, rho, epsA, gammaA, epsI, gammaI, dI, gammaD, dD, tempos = tempos)

C:\Users\biafa\Anaconda3\lib\site-packages\scipy\integrate\_ivp\common.py:32: UserWarning:

The following arguments have no effect for a chosen solver: `args`.



TypeError: diferencial() missing 13 required positional arguments: 'beta', 'theta', 'p', 'lamb', 'sigma', 'rho', 'epsA', 'gammaA', 'epsI', 'gammaI', 'dI', 'gammaD', and 'dD'

In [ ]:
plt.figure(figsize=(12,6))

# exibe os gráficos das médias
plt.plot(tempos, X.S, 'tab:blue', label='SIR: suscetíveis')
plt.plot(tempos, X.I, 'tab:red', label='SIR: infectados')
plt.plot(tempos, X.R, 'tab:green', label='SIR: recuperados')
plt.plot(tempos, X.I + X.R, 'tab:gray', label='SIR: inf. + sus.')

# informações para o gráfico
plt.xlabel('dias', fontsize=14)
plt.ylabel('número de indivíduos', fontsize=14)
plt.title('Evolução da população em cada compartimento da simulação do modelo SIR', fontsize=16)
plt.legend(loc='best', fontsize=12)
plt.show() 



In [ ]:
expostos = [np.logical_or(np.logical_and(pop_novos_infectados == 0, pop_novos_assintomaticos == 0, pop_aux == 2), expostos_novos==1)]

In [ ]:
expostos_continuam = np.select([np.logical_and(pop_novos_infectados == 0, pop_novos_assintomaticos == 0)], [np.ones(num_pop)])
expostos = np.select([np.logical_or(expostos_continuam == 1, expostos_novos == 1)], np.ones(num_pop) )

In [41]:
a = np.array([1,2,3,4,5])
b = np.array([6,7,8,9,10])

In [94]:
np.select([np.random.rand(len(a)) < 0.5], [a])

array([1, 0, 3, 4, 0])

In [71]:
np.select([np.logical_and(a < 4, b %2 == 0)], [a])

array([1, 0, 3, 0, 0])

In [69]:
np.random.rand(5)

array([0.70252813, 0.41059049, 0.18816847, 0.88813461, 0.79877876])